In [1]:
from processing.obj_preprocessing import GCS_preprocessing
from google.cloud import aiplatform as aip
from tabulate import tabulate
import json

In [13]:
PROJECT_ID = 'gcp-ccai-auto-ml-contactcenter'
REGION= "europe-west3"
REPO_NAME = "repo-demo3"
SERVICE_ACCOUNT = "944308723981-compute@developer.gserviceaccount.com"
BUCKET = "ccai-storage"
PIPELINE_NAME = "automl_pipeline"
YAML_NAME = f"{PIPELINE_NAME}.yml"
PIPELINE_ROOT = f"gs://{BUCKET}/pipeline_root/"
DISPLAY_NAME = PIPELINE_NAME.replace("_", "-")
NOTEBOOK = "automl"
DATANAME = "datasetnlp"
FOLDER = 'make_prediction'
PROJECT_ID = 'gcp-ccai-auto-ml-contactcenter'
TABLE_ID = "testdatabq"
TEXT_COLUMN = 'body_pre'
LOCATION = "europe-west3"
NUM_DOC = 1
RANDOM_SEED=123
#BQ_SOURCE = "bq://gcp-ccai-auto-ml-contactcenter.datasetnlp.stepfinalbq"

OUTPUT_FINAL = 'step_final_bq.parquet'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'

### Get Test Data from Google Storage

In [14]:
FILE_PATH = "test_file.parquet"

In [15]:
processor = GCS_preprocessing(bucket=BUCKET,
                              folder=FOLDER,
                              file_path=FILE_PATH,
                              parquet_file_name=OUTPUT_FINAL,
                              num_doc=NUM_DOC, 
                              random_seed=RANDOM_SEED,
                              project_id=PROJECT_ID,
                              dataset_id=DATANAME,
                              table_id=TABLE_ID,
                              location=LOCATION,
                              text_column=TEXT_COLUMN,
                              pipeline_root=PIPELINE_ROOT,
                              overwrite=True)


[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
instances = processor.process()
aip.Endpoint.list(filter=f'labels.notebook={NOTEBOOK}')
endpoint = aip.Endpoint.list(filter=f'labels.notebook={NOTEBOOK}')[0]
print(f"ENDPOINTS: \n{endpoint.display_name}\n")
prediction = endpoint.predict(instances = instances) # or instances = newobs

dictionary=prediction.predictions[0]
# Convert the dictionary into a list of tuples
table_data = list(zip(dictionary['classes'], dictionary['scores']))

# Print the table
print("PREDICTIONS")
print(tabulate(table_data, headers=['Class', 'Score'], tablefmt='grid'))
#print(f"PREDICTIONS: \n {prediction.predictions[0]}")

Start loading DataFrame from test_file.parquet
Start loading DataFrame from test_file.parquet

INPUT DATAFRAME
                  uri                                                url  \
0  2023-12-196637772  https://www.repubblica.it/il-gusto/2023/12/19/...   
1  2023-12-196631748  https://www.libero.it/magazine/news/grande-fra...   
2         7893333817  https://www.tgcom24.mediaset.it/lifestyle/beau...   
3         7893333963  https://www.corriere.it/lodicoalcorriere/index...   
4  2023-12-196630770  https://www.repubblica.it/tecnologia/2023/12/1...   

                                               title  \
0  Tutto il gusto dell'Abruzzo d'inverno, tra cas...   
1  GF, Varrese sotto attacco: pesanti critiche da...   
2  Natale, il make up più chic delle Feste è ispi...   
3  La rivalità Goggia-Brignone    grande romanzo ...   
4                        Tech: i personaggi del 2023   

                                                body        date      time  \
0  Tra ciaspolate, rif

### Get Test Data from Json Sample

In [ ]:
json_string = {
    "uri": "7892121222",
    "url": "https://www.ilfattoquotidiano.it/2023/12/18/auto-si-schianta-contro-suv-della-scorta-di-biden-il-tonfo-e-la-sorpresa-del-presidente-video/7386540/",
    "title": "Auto si schianta contro suv della scorta di Biden: il tonfo e la sorpresa del presidente - Video - Il Fatto Quotidiano",
    "body": "Al momento dell'impatto di un'auto contro un suv del corteo presidenziale, Joe Biden stava uscendo da un evento elettorale al quartier generale della campagna per il 2024 a Wilmington. Lo riferisce la Casa Bianca in una nota. Nel momento in cui l'auto, una berlina argentata, ha colpito un suv del corteo gli uomini del Secret Service l'hanno circondata con le pistole puntate e il conducente ha alzato le mani. Il presidente ha assistito alla scena con un'espressione sorpresa e poi è stato scortato alla sua auto e quindi nella sua residenza in Delaware, riferiscono i giornalisti al seguito.",
    "date": "2023-12-18",
    "time": "08:47:11",
    "dateTime": "2023-12-18T08:47:11Z",
    "dateTimePub": "2023-12-18T08:42:03Z",
    "lang": "ita",
    "isDuplicate": "True",
    "dataType": "news",
    "sentiment": "nan",
    "eventUri": "None",
    "relevance": "1",
    "image": "https://st.ilfattoquotidiano.it/wp-content/uploads/2023/12/18/BidenAuto.jpg",
    "authors": "F. Q.",
    "sharesFacebook": "0",
    "sourceTitle": "Il Fatto Quotidiano",
    "sourceLocationLabel": "Rome",
    "categoryLabels": "news/Politics",
    "categoryWeights": "53",
    "importanceRank": "191788",
    "alexaGlobalRank": "3336",
    "alexaCountryRank": "47"
}

# Convert dictionary to JSON string
FILE_PATH_JSON = json.dumps(json_string, indent=4)

# Print the JSON string
print(FILE_PATH_JSON)